# Quick draw classifier ✍️📖

The [Quick Draw Dataset](https://github.com/googlecreativelab/quickdraw-dataset) is a collection of 50 million drawings across 345 categories, contributed by players of the game Quick, Draw! - ***Google***.

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
# !gdown https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/simplified

In [37]:
classes = ["banana", "circle", "church", "pants", "rainbow", "sun", "cloud", "pizza", "butterfly", "penguin", "whale"]

In [38]:
len(classes)

11

In [35]:
os.mkdir("data")

FileExistsError: [Errno 17] File exists: 'data'

In [39]:
import urllib.request
base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
for c in classes:
  cls_url = c.replace('_', '%20')
  path = base+cls_url+'.npy'
  print(path)
  urllib.request.urlretrieve(path, "data/" + c + '.npy')

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/circle.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/church.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pants.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/rainbow.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/sun.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cloud.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pizza.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/butterfly.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/penguin.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/whale.npy


In [40]:
import glob

def load_data(root, vfold_ratio=0.2, max_items_per_class=35000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables 
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []
    
    #load a subset of the data to memory 
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #separate into training and testing 
    permutation = np.random.permutation(y.shape[0])
    
    x = x[permutation, :]
    y = y[permutation]

    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))
    print(vfold_size)
    X_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    X_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return X_train, y_train, X_test, y_test, class_names

In [41]:
X_train, y_train, X_test, y_test, classes = load_data("data")

77000


In [42]:
classes

['penguin',
 'cloud',
 'banana',
 'pizza',
 'sun',
 'pants',
 'rainbow',
 'whale',
 'church',
 'circle',
 'butterfly']

In [43]:
X_train.shape

(308000, 784)

In [44]:
import cv2

def preprocess(X_train, X_test):
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype("uint8")
  X_train = np.repeat(X_train, 3, -1)
  X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype("uint8")
  X_test = np.repeat(X_test, 3, -1)

  X_train = cv2.bitwise_not(X_train)
  X_test = cv2.bitwise_not(X_test)

  X_train = X_train / 255.
  X_test = X_test / 255.

  return X_train, X_test

In [45]:
X_train, X_test = preprocess(X_train, X_test)

In [46]:
X_test[0].dtype

dtype('float64')

In [72]:
from google.colab.patches import cv2_imshow
cv2_imshow(X_test[6] * 255)

ModuleNotFoundError: No module named 'google.colab'

In [71]:
X_test[0][17]

array([[1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [0.92156863, 0.92156863, 0.92156863],
       [0.00784314, 0.00784314, 0.00784314],
       [0.01568627, 0.01568627, 0.01568627],
       [0.49019608, 0.49019608, 0.49019608],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [0.62745098, 0.62745098, 0.62745098],
       [0.        , 0.        , 0.        ],
       [0.09803922, 0.09803922, 0.09803922],
       [0.36078431, 0.36078431, 0.36078431],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.

In [70]:
X_test[0][17]

array([[1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [0.92156863, 0.92156863, 0.92156863],
       [0.00784314, 0.00784314, 0.00784314],
       [0.01568627, 0.01568627, 0.01568627],
       [0.49019608, 0.49019608, 0.49019608],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [0.62745098, 0.62745098, 0.62745098],
       [0.        , 0.        , 0.        ],
       [0.09803922, 0.09803922, 0.09803922],
       [0.36078431, 0.36078431, 0.36078431],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.

In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [1]:
input_shape = (28, 28, 3)

# Build model
model_1 = tf.keras.models.Sequential([
  Input(shape=input_shape),
  Conv2D(32, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  Conv2D(64, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  MaxPool2D(2),
  Dropout(0.2),
  Conv2D(128, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  MaxPool2D(2),
  Dropout(0.2),
  Dense(256, activation="relu"),
  BatchNormalization(),
  tf.keras.layers.Flatten(),
  Dense(15, activation="softmax")
])

# Compile model
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

NameError: name 'tf' is not defined

In [2]:
model_1.fit(X_train, y_train, epochs=4, validation_data=(X_test, y_test))

NameError: name 'model_1' is not defined

In [65]:
result = model_1.predict(tf.expand_dims(X_test[2], 0))
result.argmax()

1/1 [==============================] - 0s 184ms/step


0

In [66]:
model_1.save("draw_model.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [67]:
model = tf.keras.models.load_model("draw_model.h5")

In [69]:
classes

['penguin',
 'cloud',
 'banana',
 'pizza',
 'sun',
 'pants',
 'rainbow',
 'whale',
 'church',
 'circle',
 'butterfly']